## StarTracker WCS solutions - 31-Jan-23

Craig Lage - Reworked to work with the new summit_utils code

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pandas as pd
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

# Starting again

In [ ]:
#Enter which image you want to look at
[camera, num] = ['Narrow', 102]
#[camera, num] = ['Wide', 101]
date = 20230317
seqNum = 24


year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))

path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
exp = starTrackerFileToExposure(filename)
results = runCharactierizeImage(exp, 5, 25)
filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
if camera == 'Narrow':
    # Need to loosen percentageScaleError because header SECPIX is wrong
    solverResults = solver.run(exp, filteredCat, percentageScaleError=60.0, isWideField=True)
    boresight = (1898.10, 998.47)
if camera == 'Wide':
    solverResults = solver.run(exp, filteredCat, isWideField=True)
    boresight = (1560.85, 1257.15)
newWcs = solverResults.wcs
rms_error = solverResults.rmsErrorArsec
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

hdulist = pf.open(filename)
header = hdulist[0].header
raStart = header["RASTART"]
decStart = header["DECSTART"]
azStart = header["AZSTART"]
#azStart = 360.0 - azStart # Compensating for Az sign error
if azStart > 360.0:
    azStart -= 360.0
if azStart < 0.0:
    azStart += 360.0

elStart = header["ELSTART"]
rotpa = header["ROTPA"]
rot = newWcs.getRelativeRotationToWcs(exp.getWcs()).asDegrees()
time = Time(header["DATE-OBS"], scale='tai')

altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
obsAltAz = skyLocation.transform_to(altAz)
az = obsAltAz.az.deg
el = obsAltAz.alt.deg
deltaAz = azStart - az
if deltaAz > 360.0:
    deltaAz -= 360.0
if deltaAz < 0.0:
    deltaAz += 360.0
deltaEl = elStart - el
print(f"{camera}, 202212{date:02}_{seqNum}, {deltaAz:.5f}, {deltaEl:.5f}, {rot:.5f}")

# Now find objects within the image
if camera == 'Narrow':
    radius = 0.5
    mag_range = 4.0
if camera == 'Wide':
    radius = 3.0
    mag_range = 7.5
mag_limit = 0.0
criteria = (f"region(circle,ICRS,{raSolve} {decSolve},{radius}d) & "
            f"Vmag > {mag_limit} & Vmag < {mag_limit+mag_range} & "
            "cat = HD")
results = Simbad.query_criteria(criteria)
#print(results)
xs = []
ys = []
names = []
for i in range(len(results)):
    ra = Angle(results[i][1] + 'hours')
    dec = Angle(results[i][2] + 'degrees')
    print(ra.deg, dec.deg)
    spherePoint = SpherePoint(afwAngle(ra.rad), afwAngle(dec.rad))
    point = newWcs.skyToPixel(spherePoint)
    xs.append(point.x)
    ys.append(point.y)
    name = results[i][0]
    names.append(name)
    print(point)
deltaPix = np.sqrt((point[0] - boresight[0])**2 + (point[1] - boresight[1])**2)
print(f"DeltaPix = {deltaPix:.2f} pixels")


## Now calculate the deltas from the narrow camera to the fast camera.

In [ ]:
filename = "/scratch/cslage/starTracker/startracker_2023-03-17.json"
df = pd.read_json(filename)
df = df.transpose()
row = df.loc[24]
time = Time(row['MJD'], format='mjd', scale='tai')

gam02Vol_RA = Angle(results[0][1] + 'hours')
gam02Vol_Dec = Angle(results[0][2] + 'degrees')

skyLocation = SkyCoord(gam02Vol_RA, gam02Vol_Dec)
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
gam02Vol_AltAz = skyLocation.transform_to(altAz)

# These are for the narrow camera
ra = row['Ra']
dec = row['Dec']
alt = row['Alt']
az = row['Az']

deltaRA = ra - gam02Vol_RA.deg
deltaDec = dec - gam02Vol_Dec.deg
print(f"gam02Vol Ra/Dec = ({gam02Vol_RA.deg:.5f}, {gam02Vol_Dec.deg:.5f}) ")
print(f"Narrow RA/Dec (20230317 seqNum 24) = ({ra:.5f}, {dec:.5f})")
print(f"Offset RA/Dec = ({deltaRA:.5f}, {deltaDec:.5f})")
raDecShift = np.sqrt(deltaDec**2 + (deltaRA * np.cos(Angle(dec * u.deg)))**2) * 3600.0
print(f"RaDec shift = {raDecShift:.2f} arcseconds")
print()
deltaAlt = (alt - gam02Vol_AltAz.alt.deg) * 3600.0
deltaAz = (az - gam02Vol_AltAz.az.deg) * 3600.0
print(f"gam02Vol Alt/Az = ({gam02Vol_AltAz.alt.deg:.5f}, {gam02Vol_AltAz.az.deg:.5f}) ")
print(f"Narrow Alt/Az (20230317 seqNum 24) = ({alt:.5f}, {az:.5f})")
print(f"Offset Alt/Az (arcseconds) = ({deltaAlt:.5f}, {deltaAz:.5f})")
altAzShift = np.sqrt(deltaAlt**2 + (deltaAz * np.cos(Angle(alt * u.deg)))**2)
print(f"AltAz shift = {altAzShift:.2f} arcseconds")

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

shiftedAlt = alt - deltaAlt / 3600.0
shiftedAz = az - deltaAz / 3600.0
#shiftedAlt = alt - 0.0 / 3600.0
#shiftedAz = az - 00.0 / 3600.0

altAz2 = SkyCoord(AltAz(alt=shiftedAlt*u.deg, az=shiftedAz*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz2.transform_to(ICRS)

shiftedRa = Angle(RaDec.ra.deg * u.deg)
shiftedDec = Angle(RaDec.dec.deg * u.deg)
spherePoint = SpherePoint(afwAngle(shiftedRa.rad), afwAngle(shiftedDec.rad))
point = newWcs.skyToPixel(spherePoint)
shiftX = point.x
shiftY = point.y

plt.figure(figsize=(16,16))
plt.title(f"StarTracker {camera}")
img = plt.imshow(exp.image.array,  interpolation='Nearest', cmap='gray', vmin=1, vmax=10)
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=1000, lw=2)
plt.scatter(boresight[0], boresight[1]\
            ,facecolors='r', edgecolors='r', marker='X', s=100, lw=2)
plt.scatter(shiftX, shiftY\
            ,facecolors='y', edgecolors='y', marker='X', s=100, lw=2)

for x, y, name in zip(xs, ys, names):
    plt.text(x+50,y, name, color='g', fontsize = 36)
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig(f"/home/craiglagegit/DATA/StarTracker_Labels_Simbad_{camera}_{date}_{seqNum}.pdf")

In [ ]:
#Enter which image you want to look at
[camera, num] = ['Narrow', 102]
#[camera, num] = ['Wide', 101]
date = 20230315
seqNum = 1044


year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))

path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
exp = starTrackerFileToExposure(filename)
results = runCharactierizeImage(exp, 5, 25)
filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
if camera == 'Narrow':
    # Need to loosen percentageScaleError because header SECPIX is wrong
    solverResults = solver.run(exp, filteredCat, percentageScaleError=60.0, isWideField=True)
    boresight = (1898.10, 998.47)
if camera == 'Wide':
    solverResults = solver.run(exp, filteredCat, isWideField=True)
    boresight = (1560.85, 1257.15)
newWcs = solverResults.wcs
rms_error = solverResults.rmsErrorArsec
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

hdulist = pf.open(filename)
header = hdulist[0].header
raStart = header["RASTART"]
decStart = header["DECSTART"]
azStart = header["AZSTART"]
#azStart = 360.0 - azStart # Compensating for Az sign error
if azStart > 360.0:
    azStart -= 360.0
if azStart < 0.0:
    azStart += 360.0

elStart = header["ELSTART"]
rotpa = header["ROTPA"]
rot = newWcs.getRelativeRotationToWcs(exp.getWcs()).asDegrees()
time = Time(header["DATE-OBS"], scale='tai')

altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
obsAltAz = skyLocation.transform_to(altAz)
az = obsAltAz.az.deg
el = obsAltAz.alt.deg
deltaAz = azStart - az
if deltaAz > 360.0:
    deltaAz -= 360.0
if deltaAz < 0.0:
    deltaAz += 360.0
deltaEl = elStart - el
print(f"{camera}, 202212{date:02}_{seqNum}, {deltaAz:.5f}, {deltaEl:.5f}, {rot:.5f}")

# Now find objects within the image
if camera == 'Narrow':
    radius = 0.5
    mag_range = 4.0
if camera == 'Wide':
    radius = 3.0
    mag_range = 7.5
mag_limit = 0.0
criteria = (f"region(circle,ICRS,{raSolve} {decSolve},{radius}d) & "
            f"Vmag > {mag_limit} & Vmag < {mag_limit+mag_range} & "
            "cat = HD")
results = Simbad.query_criteria(criteria)
#print(results)
xs = []
ys = []
names = []
for i in range(len(results)):
    ra = Angle(results[i][1] + 'hours')
    dec = Angle(results[i][2] + 'degrees')
    print(ra.deg, dec.deg)
    spherePoint = SpherePoint(afwAngle(ra.rad), afwAngle(dec.rad))
    point = newWcs.skyToPixel(spherePoint)
    xs.append(point.x)
    ys.append(point.y)
    name = results[i][0]
    names.append(name)
    print(point)
deltaPix = np.sqrt((point[0] - boresight[0])**2 + (point[1] - boresight[1])**2)
print(f"DeltaPix = {deltaPix:.2f} pixels")


In [ ]:
filename = "/scratch/cslage/starTracker/startracker_2023-03-15.json"
df = pd.read_json(filename)
df = df.transpose()
row = df.loc[1024]
time = Time(row['MJD'], format='mjd', scale='tai')

alf02Lib_RA = Angle(results[0][1] + 'hours')
alf02Lib_Dec = Angle(results[0][2] + 'degrees')

skyLocation = SkyCoord(alf02Lib_RA, alf02Lib_Dec)
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
alf02Lib_AltAz = skyLocation.transform_to(altAz)

# These are for the narrow camera
ra = row['Ra']
dec = row['Dec']
alt = row['Alt']
az = row['Az']

deltaRA = ra - alf02Lib_RA.deg
deltaDec = dec - alf02Lib_Dec.deg
print(f"alf02Lib Ra/Dec = ({alf02Lib_RA.deg:.5f}, {alf02Lib_Dec.deg:.5f}) ")
print(f"Narrow RA/Dec (20230315 seqNum 1044) = ({ra:.5f}, {dec:.5f})")
print(f"Offset RA/Dec = ({deltaRA:.5f}, {deltaDec:.5f})")
raDecShift = np.sqrt(deltaDec**2 + (deltaRA * np.cos(Angle(dec * u.deg)))**2) * 3600.0
print(f"RaDec shift = {raDecShift:.2f} arcseconds")
print()
deltaAlt = (alt - alf02Lib_AltAz.alt.deg) * 3600.0
deltaAz = (az - alf02Lib_AltAz.az.deg) * 3600.0
print(f"alf02Lib Alt/Az = ({alf02Lib_AltAz.alt.deg:.5f}, {alf02Lib_AltAz.az.deg:.5f}) ")
print(f"Narrow Alt/Az (20230315 seqNum 1044) = ({alt:.5f}, {az:.5f})")
print(f"Offset Alt/Az (arcseconds) = ({deltaAlt:.5f}, {deltaAz:.5f})")
altAzShift = np.sqrt(deltaAlt**2 + (deltaAz * np.cos(Angle(alt * u.deg)))**2)
print(f"AltAz shift = {altAzShift:.2f} arcseconds")

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

#shiftedAlt = alt - deltaAlt / 3600.0
#shiftedAz = az - deltaAz / 3600.0
shiftedAlt = alt - 0.0 / 3600.0
shiftedAz = az - 0.0 / 3600.0

altAz2 = SkyCoord(AltAz(alt=shiftedAlt*u.deg, az=shiftedAz*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz2.transform_to(ICRS)

shiftedRa = Angle(RaDec.ra.deg * u.deg)
shiftedDec = Angle(RaDec.dec.deg * u.deg)
spherePoint = SpherePoint(afwAngle(shiftedRa.rad), afwAngle(shiftedDec.rad))
point = newWcs.skyToPixel(spherePoint)
shiftX = point.x
shiftY = point.y

plt.figure(figsize=(16,16))
plt.title(f"StarTracker {camera}")
img = plt.imshow(exp.image.array,  interpolation='Nearest', cmap='gray', vmin=1, vmax=10)
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=1000, lw=2)
plt.scatter(boresight[0], boresight[1]\
            ,facecolors='r', edgecolors='r', marker='X', s=100, lw=2)
plt.scatter(shiftX, shiftY\
            ,facecolors='y', edgecolors='y', marker='X', s=100, lw=2)

for x, y, name in zip(xs, ys, names):
    plt.text(x+50,y, name, color='g', fontsize = 36)
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig(f"/home/craiglagegit/DATA/StarTracker_Labels_Simbad_{camera}_{date}_{seqNum}.pdf")

In [ ]:
filename = f'/scratch/cslage/starTracker/startracker_2023-03-21.json'
df = pd.read_json(filename)
df = df.transpose()
df = df.dropna()


In [ ]:
print(len(df))

In [ ]:
df.columns

In [ ]:

deltaAlts = []
deltaAzCosAlts = []
for seqNum in df.index.values.tolist():
    row = df.loc[seqNum]
    deltaAz = (row['Calculated Az'] - row['Calculated Az fast']) * 3600.0
    deltaAzCosAlt = deltaAz * np.cos(row['Calculated Alt'] * np.pi / 180.0)
    deltaAlt = (row['Calculated Alt'] - row['Calculated Alt fast']) * 3600.0
    deltaAlts.append(deltaAlt)
    deltaAzCosAlts.append(deltaAzCosAlt)
    delta = np.sqrt(deltaAlt**2 + deltaAzCosAlt**2)
    print(seqNum, deltaAlt, deltaAzCosAlt, delta)


In [ ]:
%matplotlib inline
plt.subplot(1,2,1)
plt.title("deltaAzCosAlts")
plt.hist(deltaAzCosAlts)
plt.text(-150,70,f"Median={np.median(deltaAzCosAlts):.1f}")
plt.xlim(-600, 600)
plt.xlabel("Fast-Narrow deltaAz*cos(alt)(arcseconds)")
plt.subplot(1,2,2)
plt.title("deltaAlts")
plt.hist(deltaAlts)
plt.text(-150,70,f"Median={np.median(deltaAlts):.1f}")
plt.xlim(-600, 600)
plt.xlabel("Fast-Narrow deltaAlt(arcseconds)")
plt.savefig(f'/scratch/cslage/starTracker/Fast_Camera_Offset_20230321.png')